In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.getOrCreate()
import datetime
import json

22/03/27 20:53:17 WARN Utils: Your hostname, LAPTOP-V8EALT7T resolves to a loopback address: 127.0.1.1; using 172.20.10.2 instead (on interface wifi0)
22/03/27 20:53:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/27 20:53:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#read json
# Task: Find the dependence of the price on the presented colors.
spark\
.read\
.option('header',True)\
.option('inferSchema', True)\
.json(f'/home/dmitry/zara_us_sample_data.json')\
.createOrReplaceTempView('zara_json')

In [27]:
# create df
df = spark.table('zara_json')
df.toPandas()

,_corrupt_record,availability,brand,color,condition,currency,description,images,language,mpn,name,price,scraped_at,size_list,sku,url
0,[,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,InStock,ZARA,Color Pale pink | 0219/805,NewCondition,USD,Bodysuit with sweetheart neckline and adjustab...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,128666521-966-2,SATIN EFFECT CORSET BODYSUIT,29.9,2021-10-13 01:21:25,S/M/L,128666521-966-2,https://www.zara.com/us/en/satin-effect-corset...
2,None,,ZARA,Color Brick | 9329/912,NewCondition,USD,Long dress with round neck and long sleeves. B...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,142448983-701-1,LONG PRINT DRESS,49.9,2021-10-13 01:21:25,XS/XS/ -/ /We'll let you know when the item is...,142448983-701-1,https://www.zara.com/us/en/long-print-dress-p0...
3,None,InStock,ZARAHOME,Color Steel | 2967/479,NewCondition,USD,Forged stainless steel kitchen scissors.,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,125067259-220-99,STEEL KITCHEN SCISSORS,49.9,2021-10-13 01:21:25,8 x 20 x 1 cm,125067259-220-99,https://www.zara.com/us/en/steel-kitchen-sciss...
4,None,InStock,ZARAHOME,Color Brown / Taupe | 5668/522,NewCondition,USD,Hanging organizer made of a cotton linen blend...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,135325938-737-99,LINEN BLEND STRIPED ORGANIZER,49.9,2021-10-13 01:21:25,50 x 5 x 60 cm,135325938-737-99,https://www.zara.com/us/en/linen-blend-striped...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,None,InStock,ZARAHOME,Color Natural | 2961/041,NewCondition,USD,Knife rack with a wooden structure. Flexible f...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,120292026-052-99,WOODEN KNIFE STAND,59.9,2021-10-13 01:23:58,17 x 24 x 11.2 cm,120292026-052-99,https://www.zara.com/us/en/wooden-knife-stand-...
1514,None,InStock,ZARA,Color Blue | 3649/710,NewCondition,USD,Chunky rubberized sandals. \n\nSole height: 1....,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,126358380-009-35,RUBBERIZED LOW HEEL SANDALS,39.9,2021-10-13 01:23:58,5/6/6½/7½/8/9/10/11,126358380-009-35,https://www.zara.com/us/en/rubberized-low-heel...
1515,None,,ZARA,Color Purple | 0881/122,NewCondition,USD,V-neck midi dress with full short sleeves. Con...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,108949262-611-1,EMBROIDERED TUNIC DRESS,69.9,2021-10-13 01:23:58,XS/S/M/L/XL/XXL,108949262-611-1,https://www.zara.com/us/en/embroidered-tunic-d...
1516,None,InStock,ZARA,Color various | 1202/306,NewCondition,USD,Pack of two pairs of boxers with elastic waist...,https://static.zara.net/stdstatic/1.234.0-b.45...,en-US,121014567-555-3,2 PACK OF COMBINATION PRINT BOXERS,25.9,2021-10-13 01:23:58,M/L/XL,121014567-555-3,https://www.zara.com/us/en/2-pack-of-combinati...


In [28]:
df = df.select(F.col('brand'), 
               F.col('name'),
               F.col('color'),
               F.col('size_list'),
               F.col('price'),
               F.to_date(F.col('scraped_at')).alias('date'))
df = df.na.drop()
df.toPandas()

,brand,name,color,size_list,price,date
0,ZARA,SATIN EFFECT CORSET BODYSUIT,Color Pale pink | 0219/805,S/M/L,29.9,2021-10-13
1,ZARA,LONG PRINT DRESS,Color Brick | 9329/912,XS/XS/ -/ /We'll let you know when the item is...,49.9,2021-10-13
2,ZARAHOME,STEEL KITCHEN SCISSORS,Color Steel | 2967/479,8 x 20 x 1 cm,49.9,2021-10-13
3,ZARAHOME,LINEN BLEND STRIPED ORGANIZER,Color Brown / Taupe | 5668/522,50 x 5 x 60 cm,49.9,2021-10-13
4,ZARAHOME,CHIFFON ORGANIZER WITH MOONS AND STARS,Color Vanilla | 5641/522,30 x 5 x 60 cm,35.9,2021-10-13
...,...,...,...,...,...,...
1511,ZARA,SHORT FLORAL PRINT DRESS,Color Multicolored | 9007/311,XS/S/M/L/XL,49.9,2021-10-13
1512,ZARAHOME,WOODEN KNIFE STAND,Color Natural | 2961/041,17 x 24 x 11.2 cm,59.9,2021-10-13
1513,ZARA,RUBBERIZED LOW HEEL SANDALS,Color Blue | 3649/710,5/6/6½/7½/8/9/10/11,39.9,2021-10-13
1514,ZARA,EMBROIDERED TUNIC DRESS,Color Purple | 0881/122,XS/S/M/L/XL/XXL,69.9,2021-10-13


In [29]:
# extract colors from df.color with udf function
with open('/home/dmitry/zara_us_sample_data.json','r') as json_file:
    colors_list = []
    data = json.load(json_file)
    for i in data:
        if len(i['color'].split(' ')) >= 2:
            colors_list.append(i['color'].split(' ')[1].lower())
        else:
            colors_list.append(i['color'].split(' ')[0].lower())

def colors(row):
    colors = colors_list
    for clr in colors:
        if clr in row.lower():
            return clr.title()
        else:
            continue

color_udf = F.udf(colors)

In [30]:
df = df\
.where('color is not null')\
.withColumn('product_color',color_udf(df.color))\
.withColumn('product_name', F.initcap(df.name))\
.withColumn('brand_name',  F.initcap(df.brand))

In [31]:
zara_products = df\
.select(F.col('brand_name'), 
        F.col('product_name'), 
        F.col('product_color'),
        F.col('size_list'),
        F.col('price'),
        F.col('date'))

zara_products.toPandas()

,brand_name,product_name,product_color,size_list,price,date
0,Zara,Satin Effect Corset Bodysuit,Pale,S/M/L,29.9,2021-10-13
1,Zara,Long Print Dress,Brick,XS/XS/ -/ /We'll let you know when the item is...,49.9,2021-10-13
2,Zarahome,Steel Kitchen Scissors,Steel,8 x 20 x 1 cm,49.9,2021-10-13
3,Zarahome,Linen Blend Striped Organizer,Brown,50 x 5 x 60 cm,49.9,2021-10-13
4,Zarahome,Chiffon Organizer With Moons And Stars,Vanilla,30 x 5 x 60 cm,35.9,2021-10-13
...,...,...,...,...,...,...
1511,Zara,Short Floral Print Dress,Multicolored,XS/S/M/L/XL,49.9,2021-10-13
1512,Zarahome,Wooden Knife Stand,Natural,17 x 24 x 11.2 cm,59.9,2021-10-13
1513,Zara,Rubberized Low Heel Sandals,Blue,5/6/6½/7½/8/9/10/11,39.9,2021-10-13
1514,Zara,Embroidered Tunic Dress,Purple,XS/S/M/L/XL/XXL,69.9,2021-10-13


In [32]:
# avg price by products color
products_update = zara_products\
.groupBy(F.col('brand_name'), F.col('product_color'))\
.agg(
    F.avg(F.col('price')).alias('avg_price_by_color'),
    F.count(F.col('product_name')).alias('product_cnt'))

products_update.toPandas()

,brand_name,product_color,avg_price_by_color,product_cnt
0,Zara,0210/313,19.90000,1
1,Zarahome,1204/553,29.90000,1
2,Zara,Coral,37.90000,2
3,Zara,Yellow,52.41125,8
4,Zarahome,Beige,58.42500,4
...,...,...,...,...
175,Zara,0110/237,25.90000,1
176,Zara,Pale,31.90000,3
177,Zara,Terracotta,43.90000,3
178,Zarahome,Mint,9.90000,1


In [35]:
products_update = products_update\
.where(F.col('product_cnt') >= 10)\
.select(F.col('brand_name'), 
        F.col('product_color'),
        F.col('avg_price_by_color').cast('Decimal(20,2)'),
        F.col('product_cnt'))\
.orderBy(F.col('avg_price_by_color'),F.col('product_cnt'), ascending = False)

products_update.toPandas()

,brand_name,product_color,avg_price_by_color,product_cnt
0,Zarahome,Natural,70.12,34
1,Zara,Multi-Color,67.02,11
2,Zara,Beige,66.73,16
3,Zara,Khaki,65.11,20
4,Zara,Navy,63.27,37
5,Zara,Red,61.31,20
6,Zara,Black,55.22,236
7,Zara,Brown,54.16,79
8,Zarahome,Brown,53.97,16
9,Zara,Multicolored,53.61,48


In [40]:
# find max prices by products color
max_prices = zara_products\
.orderBy(F.col('price'), ascending = False)\
.groupBy(F.col('product_color'),F.col('date'))\
.agg(F.first(F.col('product_name')).alias('mx_price_product_name'),
     F.max(F.col('price')).alias('max_price'))

max_prices.show()


+-------------+----------+---------------------+---------+
|product_color|      date|mx_price_product_name|max_price|
+-------------+----------+---------------------+---------+
|     0047/001|2021-10-13| Discovery Set Wee...|      5.9|
|     0110/102|2021-10-13|  Amalfi Sunray 90 Ml|     35.9|
|     0110/182|2021-10-13| Violet Blossom 30 Ml|      9.9|
|     0110/191|2021-10-13| Vetiver Pamplemou...|     29.9|
|     0110/237|2021-10-13| Flwr 001/alm Edp ...|     25.9|
|     0110/251|2021-10-13| Amalfi Sunray Lay...|     29.9|
|     0110/257|2021-10-13| Ebony Wood Pincea...|     25.9|
|     0110/339|2021-10-13| Sublime Epoque Ed...|     25.9|
|     0120/065|2021-10-13| Zara Gardenia 10 ...|      5.9|
|     0120/126|2021-10-13| Zara Gourmand Add...|     17.9|
|     0120/155|2021-10-13| Pink Flambé Winte...|     17.9|
|     0120/499|2021-10-13|    Deep Garden 30 Ml|      9.9|
|     0120/501|2021-10-13| Discovery Set Day...|      5.9|
|     0140/003|2021-10-13|          Femme 10 Ml|      5.

In [41]:
# join 2 df with different data 
zara_products_data = products_update\
.join(max_prices, 'product_color', 'outer')\
.where(products_update.product_cnt >= 10)\
.select(products_update.brand_name,
        products_update.product_color, 
        products_update.avg_price_by_color,
        products_update.product_cnt, 
        max_prices.mx_price_product_name,
        max_prices.max_price,
        max_prices.date)\
.orderBy(products_update.avg_price_by_color, products_update.product_cnt, ascending = False)

In [42]:
zara_products_data.toPandas()

,brand_name,product_color,avg_price_by_color,product_cnt,mx_price_product_name,max_price,date
0,Zarahome,Natural,70.12,34,Wrinkle-effect Linen Blanket,99.9,2021-10-13
1,Zara,Multi-Color,67.02,11,Multi-colored Fabric Backpack,89.9,2021-10-13
2,Zara,Beige,66.73,16,Sporty Leather Loafers,89.9,2021-10-13
3,Zara,Khaki,65.11,20,Water Repellent Puffer Coat,89.9,2021-10-13
4,Zara,Navy,63.27,37,Soft Leather Loafers,89.9,2021-10-13
5,Zara,Red,61.31,20,Mock Neck Jumpsuit,69.9,2021-10-13
6,Zara,Black,55.22,236,Wedges With Double Leather Straps,99.9,2021-10-13
7,Zara,Brown,54.16,79,Tasseled Leather Loafers,99.9,2021-10-13
8,Zarahome,Brown,53.97,16,Tasseled Leather Loafers,99.9,2021-10-13
9,Zara,Multicolored,53.61,48,Floral Print Dress,89.9,2021-10-13


22/03/27 21:37:51 WARN SparkContext: Killing executors is not supported by current scheduler.
